In [84]:
from Crypto.Cipher import AES
import random
import string
from bitstring import BitArray

# AES is based on XOR operation between key and plaintext.

# In AES, we need the same length on both the message and the key.


In [85]:
#Here i build the 1-bit-different messages

MSG1=[]
MSG2=[]

for i in range (msgCount):
    msg=""
    for j in range (15):
        msg=msg+random.choice(string.ascii_letters)    
        
                         #all the letters are the same, I only change the last letter to have 1-bit difference
    message1=msg+"b"     # b = 01100010, ends with 0
    message2=msg+"c"     # c = 01100011, ends with 1, so we have one-bit difference
    MSG1.append(message1)     
    MSG2.append(message2)
    #print(message1 + "   " + message2)        #for testing purposes, uncomment to print the 1-bit-different messages


In [86]:
def AES_ECB(msgCount,key):

    BitsChanged=[]   #Here I will store the number of bits that were different in the same position on every pair of ciphertexts

    for i in range (msgCount):
        ECB = AES.new(key, AES.MODE_ECB)
        message1 = MSG1[i]
        message2 = MSG2[i]

        ciphertext1 = ECB.encrypt(message1)
        ciphertext2 = ECB.encrypt(message2)

        c1 = BitArray(ciphertext1)
        c2 = BitArray(ciphertext2)
        #print(c1.bin)     # for testing purposes, uncomment to see the 2 binary ciphertexts
        #print(c2.bin)
        #print("\n")

        counter=0
        for j in range (128):      # Here I count the different bits in the ciphertext
            if c1.bin[j]!=c2.bin[j]:
                counter=counter + 1 
        BitsChanged.append(counter)


    average=0         #Here i find the average bit difference
    for i in BitsChanged:
        average=average+i
    average=average//msgCount

    print("AES - ECB Mode\n______________\n")
    print("Different bits in every pair of ciphertexts:\n")    
    print(BitsChanged)
    print("\nAverage bit difference:\n")  
    print(str(average) + " bits, " + str("{:.2f}".format(average/128)) + "% chance of bit-difference")
    print("\nSince it's ~50%, we can safely assume that the AES.ECB has a good Avalance Effect\n")


In [87]:
def AES_CBC(msgCount,key,IV):

    BitsChanged=[]   #Here I will store the number of bits that were different in the same position on every pair of ciphertexts

    for i in range (msgCount):
        CBC = AES.new(key, AES.MODE_CBC, IV)
        message1 = MSG1[i]
        message2 = MSG2[i]

        ciphertext1 = CBC.encrypt(message1)
        ciphertext2 = CBC.encrypt(message2)

        c1 = BitArray(ciphertext1)
        c2 = BitArray(ciphertext2)
        #print(c1.bin)     # for testing purposes, uncomment to see the 2 binary ciphertexts
        #print(c2.bin)
        #print("\n")

        counter=0
        for j in range (128):      # Here I count the different bits in the ciphertext
            if c1.bin[j]!=c2.bin[j]:
                counter=counter + 1 
        BitsChanged.append(counter)


    average=0         #Here i find the average bit difference
    for i in BitsChanged:
        average=average+i
    average=average//msgCount

    print("AES - CBC Mode\n______________\n")
    print("Different bits in every pair of ciphertexts:\n")    
    print(BitsChanged)
    print("\nAverage bit difference:\n")  
    print(str(average) + " bits, " + str("{:.2f}".format(average/128)) + "% chance of bit-difference")
    print("\nSince it's ~50%, we can safely assume that the AES.CBC has a good Avalance Effect\n")


In [88]:
msgCount = 35     # Number of messages pairs
key = 'The 16 digit key'
IV = 'This is my 16 IV'


AES_ECB(msgCount,key)
print("----------------------------------------------------------------------------------------------\n")
AES_CBC(msgCount,key,IV)

AES - ECB Mode
______________

Different bits in every pair of ciphertexts:

[67, 70, 63, 60, 64, 64, 70, 54, 65, 56, 64, 61, 67, 52, 59, 74, 65, 70, 65, 60, 56, 53, 63, 64, 77, 69, 68, 68, 56, 69, 68, 63, 59, 71, 64]

Average bit difference:

63 bits, 0.49% chance of bit-difference

Since it's ~50%, we can safely assume that the AES.ECB has a good Avalance Effect

----------------------------------------------------------------------------------------------

AES - CBC Mode
______________

Different bits in every pair of ciphertexts:

[72, 69, 74, 66, 51, 52, 79, 48, 67, 57, 54, 70, 67, 55, 59, 59, 69, 61, 73, 58, 61, 68, 60, 68, 57, 64, 68, 59, 72, 71, 62, 55, 64, 68, 66]

Average bit difference:

63 bits, 0.49% chance of bit-difference

Since it's ~50%, we can safely assume that the AES.CBC has a good Avalance Effect

